<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/SalesMail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mecab-python3
!pip install mojimoji

     |████████████████████████████████| 15.9MB 3.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/67/f0/6e/03e5c5017afc8230e19abeacad57189138207fea2f5dd71880
Successfully built mojimoji


In [0]:
import re
import MeCab
import mojimoji

def serialize(words):
    #大文字統一
    words = words.upper()

    #改行コード無効
    words = words.replace("\n","")
    
    #チルダ統一
    ser_word = ["～", "-","－","から","MAX","~~"]
    for word in ser_word:
      words = words.replace(word,"~")
    
    #スキル見合い
    ser_word = ["スキル見合い","スキル見合","ご相談","相談"]
    for word in ser_word:
      words = words.replace(word,"00")

    #英数字のみ半角統一
    words = mojimoji.zen_to_han(words, kana=False)
      
    return words

In [0]:
def tanka_pickup(str_wk):
  #======================================================
  #スキップワード削除
  #======================================================
  del_word = ["<",">","＜","＞","[","]","《","》","≪","≫","【", "】","(",")","（","）"," ","　",":","：","完全","位","くらい","ぐらい","およそ","大体","だいたい","約","前後","円","万","下さい","ください"]
  
  base_word = '単価'
  tanka_yuragi = ["単金","金額","価格","月額","予算"]

  #◆単価が含まれるであろう箇所だけ渡す
  #正規化
  str_wk = serialize(str_wk)

  #削除
  for word in del_word:
    str_wk = str_wk.replace(word,'')

  #ゆらぎ統一
  for tanka in tanka_yuragi:
    str_wk = str_wk.replace(tanka,base_word)

  #======================================================
  #分かち書き
  #======================================================
  tagger = MeCab.Tagger("-Owakati")
  list_output = (tagger.parse(str_wk)).split(' ')

  #======================================================
  #「単価」に最も近い位置で書かれている数字が含まれる文節を抽出
  #======================================================
  tanka_str = ""

  #上限範囲指定
  if list_output.count(base_word) == 1:
    if list_output[list_output.index(base_word)+1] == "~":
      tanka_str = "00~" + list_output[list_output.index(base_word)+2]

  #上限下限指定
  if list_output.count(base_word) == 1:
    if list_output[list_output.index(base_word)+2] == "~":
      tanka_str = tanka_str + "~" + list_output[list_output.index(base_word)+3]

  #上限のみ指定
  if list_output[list_output.index(base_word)+1] == "MAX":
    tanka_str ="00~" + list_output[list_output.index(base_word)+2]
    
  return tanka_str


In [11]:
str_wk = '【金　額】～MAX45万'
pickup_str = tanka_pickup(str_wk)

print(pickup_str)

#単価変数初期化
min_tanka = 0
max_tanka = 0
med_tanka = 0

#======================================================
#範囲指定単価(exp:A～B) 編集
#======================================================
tanka_range = re.findall('[0-9]{2}~[0-9]{2}',tanka_str)

#中間値算出
#if len(tanka_range) == 1:
#  tanka1 = int(tanka_range[0].split('~')[0])
#  tanka2 = int(tanka_range[0].split('~')[1])
#  med_tanka = round((tanka1+tanka2)/2)
#else:
#  med_tanka = "0"

#下限算出
if len(tanka_range) == 1:
  min_tanka = int(tanka_range[0].split('~')[0])
else:
  min_tanka = 0

#上限算出
if len(tanka_range) == 1:
  max_tanka = int(tanka_range[0].split('~')[1])
else:
  max_tanka = 0

#======================================================
#上限指定単価2(exp:70まで) 編集
#======================================================
if max_tanka == 0:
  max_specify = re.findall('[0-9]{2}まで',str_wk)

  if len(max_specify) == 1:
    max_tanka = int(max_specify[0].replace("まで",""))
  else:
    max_tanka = 0

print("MIN:" + str(min_tanka))
print("MAX:" + str(max_tanka))
#print("MED:" + str(med_tanka))

00~45
MIN:0
MAX:45
